# Task 1 - User Overview Analysis 


 Extract and Load the xDR Dataset

In [2]:
import os
import sys

# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

# Import the load_data function from your custom module
from data_processing import load_data 

In [3]:
# Load the xdr dataset
xdr_df = load_data('../data.zip', 'data/Week2_challenge_data_source(CSV).csv')

In [4]:
# Explre the first five rows
xdr_df.head()

,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
Bearer Id,,,,,,,,,,,,,,,,,,,,,
1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.0,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.0,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,NaN,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,NaN,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,NaN,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [5]:
# Let's check the size of the data
xdr_df.shape

(150001, 54)

In [12]:
# check missing data
# Assuming your dataframe is named 'df'
missing_values = xdr_df.isnull().sum()

# Print columns with missing values only
missing_columns = missing_values[missing_values > 0]
print(missing_columns.reset_index())

                                       index       0
0                                      Start       1
1                                   Start ms       1
2                                        End       1
3                                     End ms       1
4                                  Dur. (ms)       1
5                                       IMSI     570
6                              MSISDN/Number    1066
7                                       IMEI     572
8                         Last Location Name    1153
9                            Avg RTT DL (ms)   27829
10                           Avg RTT UL (ms)   27812
11                   Avg Bearer TP DL (kbps)       1
12                   Avg Bearer TP UL (kbps)       1
13               TCP DL Retrans. Vol (Bytes)   88146
14               TCP UL Retrans. Vol (Bytes)   96649
15                       DL TP < 50 Kbps (%)     754
16            50 Kbps < DL TP < 250 Kbps (%)     754
17             250 Kbps < DL TP < 1 Mbps (%)  

**Approach to Handle Missing Values:**

Handling Missing Values in Key Columns

`IMSI, MSISDN/Number, and IMEI`: These are key identifiers, and missing values here can mean lost user data. I may need to drop rows where these columns are missing, as they are essential for user-level aggregation.

`Avg RTT DL/UL (ms)`: These are performance metrics. If missing values are frequent, I might replace them with the mean/median of their respective columns.

Throughput Metrics `(DL TP and UL TP columns)`: Similarly, these columns can be filled with the mean or median values.

In [14]:
# Drop rows with missing IMSI, MSISDN/Number, IMEI, and Last Location Name (as they are key identifiers)
df_cleaned = xdr_df.dropna(subset=['IMSI', 'MSISDN/Number', 'IMEI', 'Last Location Name'])

# List of throughput-related columns that may contain NaN values
throughput_columns = ['Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
                      'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 
                      'DL TP > 1 Mbps (%)', 'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)', 
                      '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)']

# Filling NaN values with the mean for each column
df_cleaned.loc[:, throughput_columns] = df_cleaned[throughput_columns].fillna(df_cleaned[throughput_columns].mean())


In [18]:
# Fill NaN values in 'Total UL (Bytes)' and 'Total DL (Bytes)' with 0
df_cleaned.loc[:, ['Total UL (Bytes)', 'Total DL (Bytes)']] = df_cleaned[['Total UL (Bytes)', 'Total DL (Bytes)']].fillna(0)


In [19]:
df_cleaned

,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
Bearer Id,,,,,,,,,,,,,,,,,,,,,
1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.000000,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.000000,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,105.958376,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,105.958376,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,105.958376,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.304243e+19,4/29/2019 7:28,615.0,4/30/2019 0:01,407.0,59587.0,2.082014e+14,3.366865e+10,3.533251e+13,T85721A,313.000000,...,6550499.0,18003146.0,22468983.0,7149728.0,784435351.0,12724751.0,321383162.0,14890486.0,53745392.0,819016468.0
7.277826e+18,4/29/2019 7:28,451.0,4/30/2019 6:02,214.0,81230.0,2.082022e+14,3.365069e+10,3.548311e+13,D20434A,32.000000,...,16191667.0,11763428.0,17883703.0,19678161.0,526609673.0,9197207.0,3264510.0,13487416.0,57628851.0,574175259.0
7.349883e+18,4/29/2019 7:28,483.0,4/30/2019 10:41,187.0,97970.0,2.082019e+14,3.366345e+10,3.566051e+13,D10223C,27.000000,...,13877234.0,8288284.0,19350146.0,21293148.0,626893062.0,4735033.0,712180387.0,2457758.0,39135081.0,666648844.0


In [25]:
df_cleaned.dtypes

Start                                        object
Start ms                                    float64
End                                          object
End ms                                      float64
Dur. (ms)                                   float64
IMSI                                        float64
MSISDN/Number                               float64
IMEI                                        float64
Last Location Name                           object
Avg RTT DL (ms)                             float64
Avg RTT UL (ms)                             float64
Avg Bearer TP DL (kbps)                     float64
Avg Bearer TP UL (kbps)                     float64
TCP DL Retrans. Vol (Bytes)                 float64
TCP UL Retrans. Vol (Bytes)                 float64
DL TP < 50 Kbps (%)                         float64
50 Kbps < DL TP < 250 Kbps (%)              float64
250 Kbps < DL TP < 1 Mbps (%)               float64
DL TP > 1 Mbps (%)                          float64
UL TP < 10 K

Handle outliers if any


In [27]:
import numpy as np

# Ensure were are working on a copy of the DataFrame to avoid setting issues
df_cleaned = df_cleaned.copy()

# Select only numeric columns (exclude location and date)
numeric_columns = df_cleaned.select_dtypes(include=[np.number]).columns

# Clip outliers for the numeric columns
for column in numeric_columns:
    lower_bound = df_cleaned[column].quantile(0.05)
    upper_bound = df_cleaned[column].quantile(0.95)
    df_cleaned.loc[:, column] = df_cleaned[column].clip(lower=lower_bound, upper=upper_bound)


In [28]:
df_cleaned

,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
Bearer Id,,,,,,,,,,,,,,,,,,,,,
1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,241978.9375,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.000000,...,1.585461e+07,2501332.0,8.198936e+06,9.656251e+06,278082303.0,1.434415e+07,1.717444e+08,8814393.0,36749741.0,308879636.0
1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,241978.9375,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.000000,...,2.024740e+07,19111729.0,1.833841e+07,1.722713e+07,608750074.0,1.170709e+06,5.269042e+08,15055145.0,53800391.0,653384965.0
1.311448e+19,4/9/2019 17:42,49.0,4/25/2019 11:58,652.0,241978.9375,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,105.958376,...,1.972566e+07,14699576.0,1.758779e+07,6.163408e+06,229584621.0,8.332478e+05,4.106926e+08,4215763.0,27883638.0,279807335.0
1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,241978.9375,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,105.958376,...,2.138812e+07,15146643.0,1.399465e+07,1.101489e+06,799538153.0,1.084972e+07,7.490399e+08,12797283.0,43324218.0,834328345.0
1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,947.0,241978.9375,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,105.958376,...,1.525938e+07,18962873.0,1.712458e+07,1.101489e+06,527707248.0,3.529801e+06,5.507095e+08,13910322.0,38542814.0,569138589.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.304243e+19,4/29/2019 7:28,615.0,4/30/2019 0:01,407.0,59587.0000,2.082014e+14,3.366865e+10,3.533251e+13,T85721A,220.000000,...,6.550499e+06,18003146.0,2.207511e+07,7.149728e+06,784435351.0,1.272475e+07,3.213832e+08,14890486.0,53745392.0,819016468.0
7.277826e+18,4/29/2019 7:28,451.0,4/30/2019 6:02,214.0,81230.0000,2.082021e+14,3.365069e+10,3.548311e+13,D20434A,32.000000,...,1.619167e+07,11763428.0,1.788370e+07,1.967816e+07,526609673.0,9.197207e+06,4.181994e+07,13487416.0,57628851.0,574175259.0
7.349883e+18,4/29/2019 7:28,483.0,4/30/2019 10:41,187.0,97970.0000,2.082019e+14,3.366345e+10,3.566051e+13,D10223C,27.000000,...,1.387723e+07,8288284.0,1.935015e+07,2.091159e+07,626893062.0,4.735033e+06,7.121804e+08,2457758.0,39135081.0,666648844.0




Task 1.1 - Your employer wants to have an overview of the users’ behavior on those applications.   
Aggregate per user the following information in the column  
number of xDR sessions
Session duration
the total download (DL) and upload (UL) data
the total data volume (in Bytes) during this session for each application


In [20]:
 # Aggregate the required columns
agg_df = df_cleaned.groupby('Bearer Id').agg({
    'Start': 'count', # Number of xDR sessions
    'Dur. (ms)': 'sum', # Session duration
    'Total UL (Bytes)': 'sum', # Total Upload Data
    'Total DL (Bytes)': 'sum', # Total Download Data
    'Social Media UL (Bytes)': 'sum',
    'Social Media DL (Bytes)': 'sum',
    'Google UL (Bytes)': 'sum',
    'Google DL (Bytes)': 'sum',
    'Email UL (Bytes)': 'sum',
    'Email DL (Bytes)': 'sum',
    'Youtube UL (Bytes)': 'sum',
    'Youtube DL (Bytes)': 'sum',
    'Netflix UL (Bytes)': 'sum',
    'Netflix DL (Bytes)': 'sum',
    'Gaming UL (Bytes)': 'sum',
    'Gaming DL (Bytes)': 'sum',
    'Other UL (Bytes)': 'sum',
    'Other DL (Bytes)': 'sum'
    }).reset_index()

In [22]:
agg_df

,Bearer Id,Start,Dur. (ms),Total UL (Bytes),Total DL (Bytes),Social Media UL (Bytes),Social Media DL (Bytes),Google UL (Bytes),Google DL (Bytes),Email UL (Bytes),Email DL (Bytes),Youtube UL (Bytes),Youtube DL (Bytes),Netflix UL (Bytes),Netflix DL (Bytes),Gaming UL (Bytes),Gaming DL (Bytes),Other UL (Bytes),Other DL (Bytes)
0,6.917538e+18,1,24534.0,41704610.0,500721999.0,2410.0,2404741.0,2871336.0,5791591.0,806920.0,782388.0,2071526.0,6139644.0,14668354.0,19494278.0,5333340.0,466109357.0,15950724.0,670751043.0
1,6.917538e+18,1,21489.0,38693596.0,709549265.0,11936.0,2478607.0,2825198.0,3605446.0,525108.0,446376.0,18119976.0,10281221.0,10631652.0,19455048.0,2670856.0,673282567.0,3908870.0,501608458.0
2,6.917538e+18,1,27786.0,15548926.0,854465860.0,2827.0,944612.0,56392.0,10373157.0,34038.0,128003.0,4295851.0,5385159.0,1300571.0,15755839.0,8521398.0,821879090.0,1337849.0,472846860.0
3,6.917538e+18,1,15635.0,49605688.0,825786510.0,19827.0,1817239.0,3696393.0,269988.0,896670.0,3191192.0,11089528.0,12347020.0,1738176.0,2859358.0,16257481.0,805301713.0,15907613.0,24303797.0
4,6.917538e+18,1,24264.0,21983463.0,624957763.0,18928.0,1867318.0,195216.0,1689296.0,590043.0,740633.0,8401567.0,15231815.0,2817981.0,21563985.0,6992868.0,583864716.0,2966860.0,685122214.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134128,1.318654e+19,1,80024.0,35412664.0,145293140.0,38284.0,3240226.0,2271168.0,2036152.0,387548.0,2410615.0,4343114.0,12404964.0,21649273.0,11108134.0,2834548.0,114093049.0,3888729.0,695881178.0
134129,1.318654e+19,1,145291.0,36189587.0,387001634.0,48953.0,3062671.0,4001970.0,9363661.0,866373.0,2192057.0,2152449.0,22147919.0,8603105.0,21468525.0,7569327.0,328766801.0,12947410.0,371261255.0
134130,1.318654e+19,1,86399.0,35919460.0,872688792.0,42836.0,720996.0,2100839.0,1541915.0,839789.0,2315638.0,1290963.0,11879062.0,11943452.0,22596930.0,10607174.0,833634251.0,9094407.0,697260277.0
134131,1.318654e+19,1,86399.0,40877676.0,386355068.0,39905.0,2492460.0,466218.0,11318188.0,618629.0,2612190.0,16763435.0,22163800.0,8346624.0,9522397.0,1845068.0,338246033.0,12797797.0,17385489.0


In [9]:
agg_df.isnull().sum().reset_index()

,index,0
0,Bearer Id,0
1,Start,0
2,Dur. (ms),0
3,Total UL (Bytes),0
4,Total DL (Bytes),0
5,Social Media UL (Bytes),0
6,Social Media DL (Bytes),0
7,Google UL (Bytes),0
8,Google DL (Bytes),0
9,Email UL (Bytes),0




Task 1.2 - Conduct an exploratory data analysis on those data & communicate useful insights. Ensure that you identify and treat all missing values and outliers in the dataset by replacing them with the mean or any possible solution of the corresponding column.

You’re expected to report about the following using Python script and slide  :
- Describe all relevant variables and associated data types (slide). findings. 


In [ ]:
# Handle outliers


In [ ]:
# Import necessary functions
from data_preparation import prepare_data
from eda_pipeline import EDA

from user_analysis import UserAnalysis

# Load and clean data
file_path = "path_to_xdr_data.csv"
df = prepare_data(file_path)

# EDA
basic_statistics(df)
univariate_analysis(df)
bivariate_analysis(df)

# User Analysis
print("Top 10 Handsets:")
print(top_handsets(df))

print("\nTop 3 Manufacturers:")
top_manufacturers_list = top_manufacturers(df)
print(top_manufacturers_list)

for manufacturer in top_manufacturers_list.index:
    print(f"\nTop 5 Handsets for {manufacturer}:")
    print(top_handsets_per_manufacturer(df, manufacturer))

In [ ]:
# notebooks/user_overview_analysis.ipynb

import pandas as pd
from scripts.data_aggregation import aggregate_user_data
from scripts.eda import handle_missing_values, handle_outliers, basic_statistics, plot_distribution, plot_correlation_matrix
from scripts.user_overview import top_10_handsets, top_3_manufacturers, segment_users_by_decile

# Load Data
df = pd.read_csv('path_to_your_xdr_dataset.csv')

# Aggregate Data
agg_df = aggregate_user_data(df)

# Handle Missing Values
agg_df = handle_missing_values(agg_df)

# Handle Outliers
agg_df = handle_outliers(agg_df)

# Basic Statistics
print(basic_statistics(agg_df))

# Plot Distributions
plot_distribution(agg_df, 'Total DL (Bytes)')

# Correlation Matrix
plot_correlation_matrix(agg_df)
